# Converting EVI Geotiff to PNG

In [1]:
%matplotlib inline

In [2]:
import matplotlib.pyplot as plt
import matplotlib as mpl
import os
import numpy as np
import xarray as xr
import rasterio as rio

In [3]:
pth = "/Users/lsetiawan/Desktop/shared_ubuntu/APL/MODIS/data/climatology/"
fil = os.path.join(pth,"EVI_1.tif")
print(fil)

/Users/lsetiawan/Desktop/shared_ubuntu/APL/MODIS/data/climatology/EVI_1.tif


In [4]:
# Open EVI data and read values
with rio.drivers():
    with rio.open(fil) as src:
        print(src.meta)
        data = src.read(1)


{'count': 1, 'crs': {'init': u'epsg:4326'}, 'dtype': 'int16', 'affine': Affine(0.01, 0.0, -127.829404882663,
       0.0, -0.01, 49.9999999955068), 'driver': u'GTiff', 'transform': (-127.829404882663, 0.01, 0.0, 49.9999999955068, 0.0, -0.01), 'height': 4482, 'width': 6878, 'nodata': -3000.0}


In [5]:
def make_cmap(colors, position=None, bit=False):
    '''
    make_cmap takes a list of tuples which contain RGB values. The RGB
    values may either be in 8-bit [0 to 255] (in which bit must be set to
    True when called) or arithmetic [0 to 1] (default). make_cmap returns
    a cmap with equally spaced colors.
    Arrange your tuples so that the first color is the lowest value for the
    colorbar and the last is the highest.
    position contains values from 0 to 1 to dictate the location of each color.
    '''
    import matplotlib as mpl
    import numpy as np
    bit_rgb = np.linspace(0,1,256)
    if position == None:
        position = np.linspace(0,1,len(colors))
    else:
        if len(position) != len(colors):
            sys.exit("position length must be the same as colors")
        elif position[0] != 0 or position[-1] != 1:
            sys.exit("position must start with 0 and end with 1")
    if bit:
        for i in range(len(colors)):
            colors[i] = (bit_rgb[colors[i][0]],
                         bit_rgb[colors[i][1]],
                         bit_rgb[colors[i][2]])
    cdict = {'red':[], 'green':[], 'blue':[]}
    for pos, color in zip(position, colors):
        cdict['red'].append((pos, color[0], color[0]))
        cdict['green'].append((pos, color[1], color[1]))
        cdict['blue'].append((pos, color[2], color[2]))

    cmap = mpl.colors.LinearSegmentedColormap('my_colormap',cdict,256)
    return cmap

### Create a list of RGB tuples
colors = [(229,229,229),(182,165,134),(160,138,91),
          (138,111,49),(140,127,43),(142,143,37),(144,159,31),
          (146,175,25),(138,177,21),(119,165,18),(99,154,15),(80,142,12),
          (60,131,9),(41,119,6),(22,108,3),(3,97,0),(0,23,0)] # This example uses the 8-bit RGB
### Create an array or list of positions from 0 to 1.
position = [0,0.04,0.08,0.12,0.16,0.20,0.24,0.28,0.32,0.36,0.40,0.44,0.48,0.52,0.56,0.60,1]
evi_cmap = make_cmap(colors, position=position, bit=True)

In [6]:
from PIL import Image
from pylab import cm

In [7]:
# Register Colormap
cm.register_cmap(name='evi',cmap=evi_cmap)
EVI = cm.get_cmap('evi')

In [8]:
# Masking data, making no value transparent
x = np.ma.masked_where(data < 0, data)
# Normalizing data to make it from 0-1
x_normed = x / float(x.max())
print(x_normed)

# Create Image from array and apply EVI, converting np array to values of 0-255
im = Image.fromarray(EVI(x_normed, bytes=True))
# Save image out
im.save(os.path.join("/Users/lsetiawan/Desktop/", '{0}.png'.format('EVI_1')))

[[-- -- -- ..., -- -- --]
 [-- -- -- ..., -- -- --]
 [-- -- -- ..., -- -- --]
 ..., 
 [-- -- -- ..., -- 0.33178654292343385 0.6518616727433433]
 [-- -- -- ..., -- -- 0.6373881339078555]
 [-- -- -- ..., -- 0.5724229366920782 0.4421610871726881]]
